In [1]:
import sys
sys.path.append('../')

In [2]:
import json
from collections import Counter
from projection_model.models import make_model_vocab
from tqdm import tqdm_notebook as tqdm
import os
from os.path import join
from syntax_trees.syntax_tree import SyntaxTree
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
import numpy as np

### Построение модели логистической регрессии

Для каждой пары запросов требуется построить вектор фичей - количество раз, которое встречалась эта пара слов в заданном паттерне. Такие фичи возьмём для топа популярных $500$ синтаксических паттернов.

Проблема в том, что пар слов очень много. Но надо посмотреть, сколько вообще раз встречаются в текстах первые 500 паттернов.

Загрузим самые популярные паттерны, положим их в сет

In [6]:
with open('../data/popular_patterns.csv') as patterns_file:
    popular_patterns = patterns_file.readlines()
popular_patterns = [pattern[:-1] if pattern[-1] == '\n' else pattern for pattern in popular_patterns]

In [4]:
popular_patterns_set = set(popular_patterns)

In [5]:
thesaurus = ThesaurusParser("../data/RuThes", need_closure=False, verbose=True)

Загрузим также словарь с самыми популярными словами

In [15]:
vocab = make_model_vocab()

KeyboardInterrupt: 

In [7]:
vocab_keys = list(vocab.keys())

In [8]:
vocab_keys_set = set(vocab_keys)

In [9]:
del vocab

Заведём просто счётчик встречаемости троек (гипоним, гипероним, паттерн) и посмотрим, вместится ли это в память

In [10]:
DIR_PATH = "/home/loginov-ra/MIPT/HypernymyDetection/data/Lenta/texts_tagged_processed_tree"
file_list = os.listdir(DIR_PATH)
file_list = [join(DIR_PATH, filename) for filename in file_list]

In [11]:
def is_hyponym_hypernym(hypo_cand, hyper_cand):
    if hypo_cand not in thesaurus.hypernyms_dict:
        return False
    return hyper_cand in thesaurus.hypernyms_dict[hypo_cand]

In [12]:
def get_hypernymy_pairs(multitokens):
    pairs = []
    for i, hypernym_candidate in enumerate(multitokens):
        for j, hyponym_candidate in enumerate(multitokens):
            if i == j:
                continue
            if is_hyponym_hypernym(hyponym_candidate, hypernym_candidate):
                pairs.append((j, i))
    return pairs

In [14]:
feature_counter = Counter()

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                continue
            
            multitokens, main_pos = sent['multi']
            lemmas = sent['deeppavlov']
            pos = sent['pos']
            tree_info = sent['syntax']
            
            if len(multitokens) > 100:
                continue
            
            tree = SyntaxTree(empty=True)
            tree.load_from_json(tree_info)
            
            in_vocab = [token in vocab_keys_set for token in multitokens]
            
            for hypo_multi in range(len(multitokens)):
                for hyper_multi in range(len(multitokens)):
                    if hypo_multi == hyper_multi:
                        continue
                    
                    if not in_vocab[hyper_multi] or not in_vocab[hypo_multi]:
                        continue
                    
                    hypo_main, hyper_main = main_pos[hypo_multi], main_pos[hyper_multi]
                    try:
                        pattern = tree.get_syntax_pattern(hypo_main, hyper_main, pos, lemmas)
                    except:
                        continue

                    if pattern is None:
                        continue
                    pattern = ';'.join(pattern)
                    hyponym = multitokens[hypo_multi]
                    hypernym = multitokens[hyper_multi]
                    if pattern in popular_patterns_set:
                        feature_counter[(hyponym, hypernym, pattern)] += 1

In [15]:
feature_counter.most_common(10)

[(('сообщать', 'ссылка', '{}:VERB:obl:NOUN:{}'), 3960),
 (('ссылка', 'сообщать', '{}:NOUN:obl:VERB:{}'), 3960),
 (('риа', 'новость', '{}:X:appos:NOUN:{}'), 3280),
 (('сообщаться', 'сайт', '{}:VERB:obl:NOUN:{}'), 1987),
 (('сайт', 'сообщаться', '{}:NOUN:obl:VERB:{}'), 1987),
 (('миллион', 'доллар', '{}:NOUN:nmod:NOUN:{}'), 1779),
 (('доллар', 'миллион', '{}:NOUN:nmod:NOUN:{}'), 1779),
 (('сообщать', 'интерфакс', '{}:VERB:nsubj:NOUN:{}'), 1777),
 (('интерфакс', 'сообщать', '{}:NOUN:nsubj:VERB:{}'), 1777),
 (('премьер', 'министр', '{}:NOUN:appos:NOUN:{}'), 1683)]

In [21]:
pair_pattern_features = dict()

for key, cnt in tqdm(feature_counter.items()):
    hyponym, hypernym, pattern = key
    
    if hyponym not in pair_pattern_features:
        pair_pattern_features[hyponym] = dict()
    
    if hypernym not in pair_pattern_features[hyponym]:
        pair_pattern_features[hyponym][hypernym] = Counter()
        
    pair_pattern_features[hyponym][hypernym][pattern] += 1

Файл с такими встречаемостями был записан, восстановим его

In [7]:
with open('freq_features1000.json') as features_file:
    pair_pattern_features = json.load(features_file)

Сгенерируем сразу вектора встречаемости паттернов

In [8]:
vectorized_pattern_features = dict()

for hyponym in tqdm(pair_pattern_features.keys()):
    vectorized_pattern_features[hyponym] = dict()
    for hypernym, pattern_ctr in pair_pattern_features[hyponym].items():
        cnt_list = []
        for pattern in popular_patterns:
            if pattern in pattern_ctr:
                cnt_list.append(pattern_ctr[pattern])
            else:
                cnt_list.append(0)
        vectorized_pattern_features[hyponym][hypernym] = cnt_list

_______________________

Теперь требуется собственно обучить логистическую регрессию. Для этого надо собрать обучающую выборку и засунуть в `sklearn`. Обучение составим так же с помощью случайного майнинга негативных примеров.

In [9]:
X = []
y = []

In [10]:
n_negative = 2

for hyponym, hypernyms in tqdm(thesaurus.hypernyms_dict.items()):
    if hyponym not in vectorized_pattern_features:
        continue
    for hypernym in hypernyms:
        if hypernym not in vectorized_pattern_features[hyponym]:
            continue
        X.append(vectorized_pattern_features[hyponym][hypernym])
        y.append(1)
        
        for i in range(n_negative):
            all_pairs = list(vectorized_pattern_features[hyponym].keys())
            neg_hypernym = np.random.choice(all_pairs)
            X.append(vectorized_pattern_features[hyponym][neg_hypernym])
            y.append(0)

In [11]:
del vectorized_pattern_features

In [13]:
import gc
gc.collect()

0

In [14]:
len(y)

25161

Видим, что примеров не очень много, поскольку в тезаурусе всё же есть довольно редкие пары гипоним-гипероним, и в тексте они не встречались. Поскольку на такие пары смотреть и не хочется, оставим то, что есть.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
len(X_train), len(X_test)

(20128, 5033)

In [18]:
lr_model = LogisticRegression(solver='lbfgs', max_iter=2000)

In [25]:
param_grid = {
    'C': np.logspace(-4, 2, 10),
    'penalty': ['l1', 'l2'],
    'solver': ['saga'],
    'max_iter': [300]
}

clf = GridSearchCV(lr_model, param_grid, scoring='accuracy', verbose=10, n_jobs=4)

In [26]:
clf.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/loginov-ra/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  5.7min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  7.9min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 14.5min
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed: 16.8min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed: 16.8min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=2000, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=4,
             param_grid={'C': array([1.00000000e-04, 4.64158883e-04, 2.15443469e-03, 1.00000000e-02,
       4.64158883e-02, 2.15443469e-01, 1.00000000e+00, 4.64158883e+00,
       2.15443469e+01, 1.00000000e+02]),
                         'max_iter': [300], 'penalty': ['l1', 'l2'],
                         'solver': ['saga']},
           

In [27]:
best_model = clf.best_estimator_

In [32]:
clf.best_params_

{'C': 1.0, 'max_iter': 300, 'penalty': 'l2', 'solver': 'saga'}

In [29]:
predictions = best_model.predict(X_test)

In [30]:
accuracy_score(y_test, predictions)

0.7383270415259289

Мы видим, что качество довольно хорошее при наличии таких пар в текстах

In [31]:
best_model.max_iter = 2000
best_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

Сохраним эту модель

In [33]:
import pickle

In [34]:
pickle.dump(best_model, open('../data/lr500.bin', 'wb'))

In [35]:
best_model.coef_[0][-10:]

array([ 0.85722543,  0.99021595, -0.66331773,  0.84732681,  1.24377586,
        0.84059946,  0.        ,  0.        ,  0.54562648,  0.28759792])